In [9]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
from selenium import webdriver
import pandas as pd
import warnings
import random
import timeit

warnings.filterwarnings(action='ignore')

In [10]:
# 페이지
pages = np.arange(1, 11)

# 카테고리 번호
# 반소매 티셔츠 : 001001
# 긴소매 디셔츠 : 001010
# 민소매 티셔츠 : 001011

num = "001001"


def clothes_num(num, pages):
    url_head = """https://www.musinsa.com/category/{num}?d_cat_cd={num}&brand=&rate=&page_kind=search&list_kind=small
    &sort=sale_high&sub_sort=1y&page={page}&display_cnt=90&sale_goods=&group_sale=&kids=N&ex_soldout=&color=&price1=&price2=&
    exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure="""
    clothes_num = []

    for page in pages:
        # 만약 페이지가 있다면 => 카테고리 내 페이지가 pages보다 적은 경우 예외처리
        try :
            url = url_head.format(num=num, page=page)
            r = requests.get(url)
            content = r.content
            soup = BeautifulSoup(content, "html.parser")
            items = soup.select("div > ul > li.li_box")
            for item in items:
                clothes_num.append(item["data-no"])
        except :
            pass

    return clothes_num


# 다 리스트 안에 저장 선형적으로 저장
num_li = clothes_num(num, pages)

num_li

['1382658',
 '1848166',
 '996177',
 '2086653',
 '996178',
 '1884943',
 '1420730',
 '903340',
 '1815503',
 '1841764',
 '1417691',
 '2034137',
 '996497',
 '1939099',
 '1886950',
 '1685307',
 '2442409',
 '1921901',
 '1503352',
 '2453556',
 '345889',
 '2186292',
 '1236364',
 '2035287',
 '2402005',
 '1388775',
 '1860490',
 '1899755',
 '1779737',
 '1793420',
 '1911516',
 '792918',
 '996498',
 '987149',
 '2329824',
 '2015684',
 '1807072',
 '1862477',
 '1846930',
 '751030',
 '1845485',
 '2471760',
 '1648564',
 '324416',
 '1827193',
 '2050094',
 '1417692',
 '1893766',
 '2376229',
 '1886956',
 '1820701',
 '2479911',
 '1388776',
 '2122566',
 '1417715',
 '1943792',
 '2401885',
 '2368712',
 '2274644',
 '2409894',
 '747950',
 '538899',
 '1827196',
 '1403092',
 '1375522',
 '1882396',
 '1454812',
 '470357',
 '1793413',
 '1816729',
 '1801721',
 '2398381',
 '2391261',
 '1845484',
 '2396221',
 '1005680',
 '748056',
 '1763157',
 '1728662',
 '903341',
 '2453552',
 '2462040',
 '1763164',
 '747941',
 '178264

In [11]:
def coordi_processing(data):

    coordi_series = pd.Series(data)
    coordi_list = ["아메리칸 캐주얼", "캐주얼", "시크", "댄디", "포멀", "걸리시", "골프", "홈웨어", "레트로", "로맨틱", "스포츠", "스트릿"]
    coordi_TF = coordi_series.isin(coordi_list)

    a = []
    for i in np.arange(len(coordi_TF)):
        if coordi_TF[i] == True:
            a.append(data[i])

    if len(a) == 0:
        a = ""
    else:
        a = list(set(a))

    return a

def season_processing(data, season_find):

    if data == "시즌":
        season = re.sub(" +|\n", " ", season_find).strip()
    else:
        season = ""

    return season

In [14]:
# 상품을 담을 리스트 => 전역변수
clothes_li = []

def crawling(num_li):
    # 크롬 드라이버 객체 생성
    driver = webdriver.Chrome("/Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/프로젝트/Cordi_RecSys/crawling/chromedriver")

    driver.implicitly_wait(3)

    # beautifulsoup의 select 태그는 조건에 맞는 여러 개를 가져옴
    for idx, num in enumerate(num_li):

        # 상품 코드 바꿔가면서 페이지 순회 => 상품 조회
        url = f'https://www.musinsa.com/app/goods/{num}'

        try :
            driver.get(url)

            # 1초에서 2초 사이 랜덤하게 휴식
            driver.implicitly_wait(random.randint(1, 2))

            # 바뀐 url 정보를 parsing해서 soup변수에 저장
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 상품 이름
            name = soup.select("div > div > span > em")[0].text
            # 대분류
            main_category = soup.select("div > div > p.item_categories > a")[0].text
            # 중분류
            sub_category = soup.select("div > div > p.item_categories > a")[1].text
            # 브랜드
            brand = soup.select("div > ul > li > p > strong > a")[0].text
            # 품번
            number = soup.select_one("div > ul > li > p > strong").text.split(" / ")[1]
            # 해시태그
            tags = [re.sub('[#]', '', s.text) for s in soup.select("li > p > a.listItem")]
            # 가격
            price = re.sub(" +|\n", "", soup.select("#goods_price")[0].text)
            # 평점
            rating = soup.select("span.prd-score__rating")[0].text
            # 후기 개수
            rating_num = soup.select("span.prd-score__review-count")[0].text
            rating_num = re.sub(r'[^0-9]', '', rating_num)
            # 시즌
            season_word = soup.select("a.ui-toggle-btn")[0].text.split(" ")[0]
            season_find = soup.select("li > p.product_article_contents > strong")[1].text
            season = season_processing(season_word, season_find)

            # 성별
            gender = [g.text for g in soup.select("li > p.product_article_contents > span.txt_gender > span")]

            # 좋아요 개수
            like = soup.select("li > p > span.prd_like_cnt")[0].text
            # 조회수
            view = soup.select("#pageview_1m")[0].text
            # 누적판매
            sale = soup.select("#sales_1y_qty")[0].text
            # 코디 정보
            coordi = [s.text for s in soup.select("p.style_txt")]
            coordi = coordi_processing(coordi)

            # ~ 18세
            age18 = soup.select("span.bar_num")[0].text
            # 19 ~ 23세
            age19_23 = soup.select("span.bar_num")[1].text
            # 24 ~ 28세
            age24_28 = soup.select("span.bar_num")[2].text
            # 29 ~ 33세
            age29_33 = soup.select("span.bar_num")[3].text
            # 34 ~ 39세
            age34_39 = soup.select("span.bar_num")[4].text
            # 40 ~
            age40 = soup.select("span.bar_num")[5].text
            # 남성 구매 비율
            man = soup.select("dd.label_info_value")[0].text
            # 여성 구매 비율
            woman = soup.select("dd.label_info_value")[1].text
            # 상품 사진
            img = 'https:' + soup.select("#detail_bigimg > div.product-img > img")[0]['src']

            # 옷 아이템 객체
            clothes = [name, main_category, sub_category, brand, number, tags, price, rating, rating_num, season, gender,
                       like, view, sale, coordi, age18, age19_23, age24_28, age29_33, age34_39, age40, man, woman, img]

            # 옷 리스트에 아이템 객체 추가
            clothes_li.append(clothes)

            print(f'{idx}번 | codenum : {num}, : {clothes} \n{"*" * 20}')

        except :
            pass

    driver.quit()


In [15]:
# 크롤링 실행해서 저장
crawling(num_li)

0번 | codenum : 1382658, : ['레이어드 크루 넥 반팔 티셔츠_긴 기장 [화이트]', '상의', '반소매 티셔츠', 'MUSINSA STANDARD', 'MMJTS113-WH', ['기본티셔츠', '라운드넥', 'MU-T', '레귤러핏', '베이직티', '무지티', '무탠픽업', '서머아이템기획전'], '13,900원', '4.7', '44302', '', ['남'], '69,869', '1.5만 회 이상', '14.3만 개 이상', ['캐주얼', '스포츠', '댄디'], '42%', '23%', '14%', '8%', '6%', '7%', '80%', '20%', 'https://image.msscdn.net/images/goods_img/20200402/1382658/1382658_5_500.jpg?t=20210705133843'] 
********************
1번 | codenum : 1848166, : ['에센셜 쿨 코튼 2-PACK 티셔츠', '상의', '반소매 티셔츠', 'COVERNAT', 'CO0000STE1BK', ['반팔티셔츠', '티셔츠', '반팔티', '오버핏반팔', '에센셜라인', '쿨코튼', '썸머컬렉션', '로고티셔츠', '입시덕후', '깡스타일리스트PICK'], '49,000원', '4.8', '32541', '2022 S/S', ['남', '여'], '126,528', '41.9만 회 이상', '12.6만 개 이상', ['걸리시', '아메리칸 캐주얼', '캐주얼'], '37%', '21%', '19%', '11%', '4%', '8%', '64%', '36%', 'https://image.msscdn.net/images/goods_img/20210316/1848166/1848166_11_500.jpg?t=20220404173105'] 
********************
2번 | codenum : 996177, : ['릴렉스 핏 크루 넥 반팔 티셔츠 [화이트]', '상의', '반소매 티셔츠', 'MU

19번 | codenum : 2453556, : ['TSHIRT FLOWERMARDI_IVORY NAVY', '상의', '반소매 티셔츠', 'MARDI MERCREDI', '430770', ['그래픽'], '42,000원', '4.9', '3976', '2022 S/S', ['여'], '51,021', '16.3만 회 이상', '2.4만 개 이상', ['캐주얼'], '35%', '28%', '17%', '7%', '4%', '9%', '18%', '82%', 'https://image.msscdn.net/images/goods_img/20220329/2453556/2453556_1_500.jpg?t=20220331173150'] 
********************
20번 | codenum : 345889, : ['[패키지] 더블코튼 무지반팔  패키지 KHST1011', '상의', '반소매 티셔츠', 'FP142', '565175', ['레이어드', '면티', '무지반팔', '반팔티', '티셔츠', '페플무지', '오버핏반팔', '무지'], '31,800원', '4.8', '6556', '', ['남'], '17,935', '5.9만 회 이상', '2.3만 개 이상', ['골프', '댄디', '캐주얼'], '25%', '27%', '23%', '14%', '5%', '6%', '85%', '15%', 'https://image.msscdn.net/images/goods_img/20160429/345889/345889_12_500.jpg?t=20220415100101'] 
********************
21번 | codenum : 2186292, : ['[2PACK] 베이식 무지 레이어드 반팔 티셔츠 일반 기장', '상의', '반소매 티셔츠', 'PRINTSTAR', '2-085', ['반팔', '반팔티셔츠', '반팔티', '티셔츠', '레이어드', '이너', '레이어드룩'], '20,000원', '4.8', '5071', '병행수입', ['남'], '

39번 | codenum : 751030, : ['라인 서퍼맨 티셔츠 화이트', '상의', '반소매 티셔츠', 'COVERNAT', 'CO1804SL20WH', ['그래픽', '반팔', '서퍼맨', '오버핏', '트로피컬', '썸머컬렉션', '그래픽티셔츠'], '39,000원', '4.8', '22884', '2020 S/S', ['남', '여'], '38,708', '6.7만 회 이상', '1.7만 개 이상', ['캐주얼'], '32%', '20%', '17%', '12%', '7%', '12%', '50%', '50%', 'https://image.msscdn.net/images/goods_img/20180406/751030/751030_5_500.jpg?t=20210426110059'] 
********************
40번 | codenum : 1845485, : ['더블 엔젤 와펜 반팔 티셔츠 블랙', '상의', '반소매 티셔츠', 'WHATITISNT', 'MG2200ST06BK', ['반팔티셔츠', '반팔티', '티셔츠', '반팔', '엔젤와펜', '동행세일', '로고/레터링', '더블엔젤'], '39,000원', '4.8', '8302', '2022 S/S', ['남', '여'], '28,321', '3.7만 회 이상', '1.7만 개 이상', ['캐주얼'], '42%', '22%', '14%', '6%', '3%', '13%', '54%', '46%', 'https://image.msscdn.net/images/goods_img/20210315/1845485/1845485_2_500.jpg?t=20220329162400'] 
********************
41번 | codenum : 2471760, : ['(22SS) 2 TONE ARCH TEE ROYAL BLUE', '상의', '반소매 티셔츠', 'YALE', 'YC2SS0041RB', ['그래픽', '최현욱착용'], '39,000원', '4.9', '9808', '2022 S

59번 | codenum : 2409894, : ['[COOL COTTON] 2PACK SMALL ARCH TEE WHITE / BLACK', '상의', '반소매 티셔츠', 'YALE', 'YC1SS1011BK', ['로고', '오버핏반팔', '반팔티', '오버핏', '반팔티셔츠', '스포츠웨어', '패키지/셋업'], '78,000원', '4.9', '2409', '2022 S/S', ['남', '여'], '13,085', '7.2만 회 이상', '1.3만 개 이상', ['캐주얼'], '29%', '25%', '24%', '11%', '4%', '7%', '53%', '47%', 'https://image.msscdn.net/images/goods_img/20220310/2409894/2409894_11_500.jpg?t=20220520144316'] 
********************
60번 | codenum : 747950, : ['[Mmlg] 19MG HF-T (WHITE)', '상의', '반소매 티셔츠', 'MMLG', 'MMLGT123', ['로고', '반팔티', '반팔티셔츠', '팔칠엠엠', '하프티', '87MM', '로고티셔츠', '특별기획전', 'EVERYDAYWEAR', '그래픽'], '42,000원', '4.8', '15709', '', ['남', '여'], '47,509', '5.8만 회 이상', '1.2만 개 이상', ['걸리시', '캐주얼'], '33%', '29%', '21%', '7%', '3%', '7%', '43%', '57%', 'https://image.msscdn.net/images/goods_img/20180403/747950/747950_1_500.jpg?t=20220321112716'] 
********************
61번 | codenum : 538899, : ['아치 로고 티셔츠 화이트', '상의', '반소매 티셔츠', 'COVERNAT', 'CO0000ST02WH', ['반팔티', '오버핏', '티셔

79번 | codenum : 903341, : ['빅 트위치 로고 티셔츠 다크 블루', '상의', '반소매 티셔츠', 'LEE', 'LE2202ST01DB', ['로고', '반팔티셔츠', '루즈핏', '남녀공용', '반팔', '트위치', '그래픽'], '35,000원', '4.8', '47389', 'ALL ALL', ['남'], '24,347', '2만 회 이상', '1만 개 이상', ['캐주얼'], '42%', '26%', '15%', '5%', '3%', '9%', '38%', '62%', 'https://image.msscdn.net/images/goods_img/20181112/903341/903341_4_500.jpg?t=20210528101656'] 
********************
80번 | codenum : 2453552, : ['TSHIRT FLOWERMARDI_BLACK CREAM', '상의', '반소매 티셔츠', 'MARDI MERCREDI', '430767', ['그래픽'], '42,000원', '4.9', '1769', '2022 S/S', ['여'], '20,458', '4.5만 회 이상', '1만 개 이상', '', '29%', '26%', '20%', '9%', '5%', '11%', '14%', '86%', 'https://image.msscdn.net/images/goods_img/20220329/2453552/2453552_1_500.jpg?t=20220331173345'] 
********************
81번 | codenum : 2462040, : ['PRDA DENIM PATCH CREWNECK TEE WHITE', '상의', '반소매 티셔츠', 'PRENDA', 'PRTTSURS03WC2', ['티셔츠추천', '반팔티셔츠', '여자반팔티', '반팔티', '반팔', '크롭'], '45,000원', '4.8', '2426', '2022 S/S', ['여'], '32,937', '12만 회 이상', '9.6천

...

801번 | codenum : 1386488, : ['[16수] 오아시스 피그먼트 자수 반팔 딥챠콜', '상의', '반소매 티셔츠', 'CPGN STUDIO', 'SP020320', ['그래픽', '남녀공용', '반팔티', '오버핏', '자수', '피그먼트'], '45,000원', '4.7', '1460', '2020 S/S', ['남'], '5,935', '2.6천 회 이상', '800개 이상', '', '23%', '25%', '23%', '15%', '8%', '6%', '72%', '28%', 'https://image.msscdn.net/images/goods_img/20200406/1386488/1386488_1_500.jpg?t=20200406153006'] 
********************
802번 | codenum : 572126, : ['[패키지] [UNISEX] 45수 특양면 옆트임 오버핏 반팔티 (화이트/블랙)', '상의', '반소매 티셔츠', 'TEEAZ', 'TMY100HR-SET-2', ['레이어드', '반팔', '반팔티', '오버핏', '티셔츠', '오버핏티셔츠', '레이어드티셔츠'], '42,000원', '4.6', '1291', '', ['남'], '1,657', '600회 이상', '800개 이상', '', '24%', '11%', '25%', '13%', '13%', '14%', '83%', '17%', 'https://image.msscdn.net/images/goods_img/20170607/572126/572126_1_500.jpg?t=20170607144634'] 
********************
803번 | codenum : 2280742, : ['[Mmlg] 19MG PILE HF-T (ULTRA MARINE)', '상의', '반소매 티셔츠', 'MMLG', 'MMLGBB1T033', ['반팔티', '남자반팔티', '여자반팔티', '반팔', '반팔티셔츠', '티셔츠추천', '티셔츠', '특별기획전', '

821번 | codenum : 2380602, : ['939 LOGO T-SHIRTS #02 (WHITE)', '상의', '반소매 티셔츠', 'ARCHIVE BOLD', 'AB_22SS_TS_002_WH', [], '39,000원', '4.8', '1274', '2022 S/S', ['남', '여'], '2,586', '9.1천 회 이상', '800개 이상', ['캐주얼'], '49%', '30%', '9%', '2%', '1%', '9%', '28%', '72%', 'https://image.msscdn.net/images/goods_img/20220223/2380602/2380602_1_500.jpg?t=20220314105734'] 
********************
822번 | codenum : 1466672, : ['[패키지] 오버핏 쿨 마이크로아이스 라운드 티셔츠 2P SET 16COLOR', '상의', '반소매 티셔츠', 'GAINSBORO', 'GB20084', ['반팔', '패키지', '운동복', '여름반팔티', '무지티', '레이어드', '루즈핏반팔'], '58,000원', '4.7', '385', 'ALL S/S', ['남'], '774', '1000회 이상', '700개 이상', '', '12%', '18%', '30%', '19%', '13%', '8%', '77%', '23%', 'https://image.msscdn.net/images/goods_img/20200528/1466672/1466672_8_500.jpg?t=20210720144733'] 
********************
823번 | codenum : 2329301, : ['E.P T-SHIRT - WHITE', '상의', '반소매 티셔츠', 'NOMANUAL', 'NM12TS08M1WH', ['반팔티', '티셔츠', '반팔티셔츠', '오버핏반팔', '반팔', '22S/S컬렉션', '그래픽'], '46,000원', '4.9', '607', '2022 S/S', ['

841번 | codenum : 2572338, : ['하프 집업 반팔 스웻셔츠 쇼츠 세트 [4 Colors]', '상의', '반소매 티셔츠', 'SUPERSUB', 'DIEJLKJDIUO', ['하프집업', '반바지', '쇼츠', '트레이닝', '트레이닝세트', '셋업', '아노락'], '79,000원', '4.9', '73', '2022 S/S', ['남'], '2,196', '2.8만 회 이상', '700개 이상', '', '29%', '21%', '17%', '12%', '9%', '12%', '43%', '57%', 'https://image.msscdn.net/images/goods_img/20220520/2572338/2572338_1_500.jpg?t=20220523090633'] 
********************
842번 | codenum : 2458750, : ['[22HS] 로고 크롭 반팔 티셔츠_BLACK', '상의', '반소매 티셔츠', 'CITY BREEZE', 'CTE2TS01BK', [], '42,000원', '4.8', '597', '2022 S/S', ['여'], '4,768', '1만 회 이상', '700개 이상', ['홈웨어', '시크', '스포츠', '캐주얼'], '20%', '46%', '24%', '5%', '2%', '3%', '4%', '96%', 'https://image.msscdn.net/images/goods_img/20220331/2458750/2458750_1_500.jpg?t=20220411133443'] 
********************
843번 | codenum : 2479602, : ['UNIVERSITY HANDSOME DAN TEE BLACK', '상의', '반소매 티셔츠', 'YALE', 'YA2SS0041BK', ['여름반팔티', '티셔츠', '반팔티', '반팔', '반팔티셔츠', '여름반팔', '반팔티추천'], '42,000원', '4.9', '3217', '2021 S/S', [

861번 | codenum : 2442497, : ['(잔망루피) 사회생활 만렙 루피 티셔츠(WHITE)_SPRLC25C01', '상의', '반소매 티셔츠', 'SPAO', 'SPRLC25C01', ['잔망루피', '스파오잔망루피', '스파오X잔망루피', '반팔티', '반팔', '티셔츠', '반팔티셔츠'], '25,900원', '4.9', '866', '2022 S/S', ['남', '여'], '1,056', '9.2천 회 이상', '700개 이상', '', '53%', '21%', '13%', '6%', '2%', '5%', '51%', '49%', 'https://image.msscdn.net/images/goods_img/20220324/2442497/2442497_18_500.jpg?t=20220531144508'] 
********************
862번 | codenum : 1921729, : ['[16수] Run Smile 자수 반팔 화이트', '상의', '반소매 티셔츠', 'CPGN STUDIO', 'SL213C04', ['그래픽', '남녀공용', '반팔티', '오버핏', '자수'], '45,000원', '4.8', '325', '2021 S/S', ['여'], '3,956', '3.6천 회 이상', '700개 이상', '', '25%', '28%', '18%', '8%', '7%', '14%', '35%', '65%', 'https://image.msscdn.net/images/goods_img/20210427/1921729/1921729_1_500.jpg?t=20210427093139'] 
********************
863번 | codenum : 2472352, : ['ATH TEE(SURF CLUB)_ORANGE', '상의', '반소매 티셔츠', 'OUTSTANDING', 'OSTD214', ['반팔티', '티셔츠', '반팔', '여름반팔', '오버핏', '로고티셔츠', '그래픽', '핏더사이즈PICK'], '42,000원

881번 | codenum : 1431737, : ['릴렉스 핏 크루 넥 반팔 티셔츠 [더스티 그린]', '상의', '반소매 티셔츠', 'MUSINSA STANDARD', 'MITS0005-UG', ['기본티', '라운드넥', '루즈핏', '면티셔츠', '무지티', '반팔티', 'MU-T', '반소매티셔츠'], '11,900원', '4.8', '159302', '', ['남'], '476', '1.6천 회 이상', '700개 이상', ['댄디', '캐주얼'], '15%', '15%', '22%', '18%', '15%', '15%', '79%', '21%', 'https://image.msscdn.net/images/goods_img/20200506/1431737/1431737_4_500.jpg?t=20201109160354'] 
********************
882번 | codenum : 1907862, : ['292513 ribbed knit T-shirt(BLACK)', '상의', '반소매 티셔츠', 'ROMANTICPIRATES', '20210220', ['무지티', '반팔티', '반팔티셔츠', '라운드티', '기본티', '베이직티셔츠'], '33,500원', '4.8', '1546', '2022 S/S', ['남'], '952', '3.2천 회 이상', '700개 이상', ['댄디'], '16%', '27%', '31%', '18%', '5%', '3%', '94%', '6%', 'https://image.msscdn.net/images/goods_img/20210419/1907862/1907862_3_500.jpg?t=20220614153105'] 
********************
883번 | codenum : 2477490, : ['FANCY FLOWER TEE_white', '상의', '반소매 티셔츠', 'NOTHINGISBETTERTHANME', 'i24', ['남자반팔티', '로고티셔츠', '여름반팔티', '티셔츠추천', '오버핏

In [16]:
clothes_li

[['레이어드 크루 넥 반팔 티셔츠_긴 기장 [화이트]',
  '상의',
  '반소매 티셔츠',
  'MUSINSA STANDARD',
  'MMJTS113-WH',
  ['기본티셔츠', '라운드넥', 'MU-T', '레귤러핏', '베이직티', '무지티', '무탠픽업', '서머아이템기획전'],
  '13,900원',
  '4.7',
  '44302',
  '',
  ['남'],
  '69,869',
  '1.5만 회 이상',
  '14.3만 개 이상',
  ['캐주얼', '스포츠', '댄디'],
  '42%',
  '23%',
  '14%',
  '8%',
  '6%',
  '7%',
  '80%',
  '20%',
  'https://image.msscdn.net/images/goods_img/20200402/1382658/1382658_5_500.jpg?t=20210705133843'],
 ['에센셜 쿨 코튼 2-PACK 티셔츠',
  '상의',
  '반소매 티셔츠',
  'COVERNAT',
  'CO0000STE1BK',
  ['반팔티셔츠',
   '티셔츠',
   '반팔티',
   '오버핏반팔',
   '에센셜라인',
   '쿨코튼',
   '썸머컬렉션',
   '로고티셔츠',
   '입시덕후',
   '깡스타일리스트PICK'],
  '49,000원',
  '4.8',
  '32541',
  '2022 S/S',
  ['남', '여'],
  '126,528',
  '41.9만 회 이상',
  '12.6만 개 이상',
  ['걸리시', '아메리칸 캐주얼', '캐주얼'],
  '37%',
  '21%',
  '19%',
  '11%',
  '4%',
  '8%',
  '64%',
  '36%',
  'https://image.msscdn.net/images/goods_img/20210316/1848166/1848166_11_500.jpg?t=20220404173105'],
 ['릴렉스 핏 크루 넥 반팔 티셔츠 [화이트]',
  '상의',
  '반소매 티셔

In [18]:
# 데이터 프레임 생성
df = pd.DataFrame(clothes_li,
                  columns=["name", "main_category", "sub_category", "brand", "number", "tags","price", "rating", "rating_num", "season", "gender",
                           "like", "view", "sale", "coordi", "age18","age19_23", "age24_28", "age29_33", "age34_39", "age40", "man", "woman", "img"])

# csv 저장
df.to_csv('short_sleeve_t-shirt.csv', index=True)